# Test models

In [1]:
from datasets import PlacesAudio
import sys
from opts import get_arguments
from utils import util as u
from torch.utils.data import Dataset, DataLoader, dataset
from torch.autograd import Variable
import time
import torch
from tqdm import tqdm


In [2]:
def validate(val_loader, model, criterion, device, epoch, args):
    batch_time = u.AverageMeter()
    losses = u.AverageMeter()
    acc_a_vAverage = u.AverageMeter()
    acc_v_aAverage = u.AverageMeter() 

    tic = time.time()

    model.eval()
    
    with torch.no_grad():
        end = time.time()
        for idx, (image, spec, audio, name, im) in tqdm(enumerate(val_loader), total=len(val_loader)):

            spec = Variable(spec).to(device, non_blocking=True)
            image = Variable(image).to(device, non_blocking=True)
            B = image.size(0)

            imgs_out, auds_out = model(image.float(), spec.float(), args, mode='val')
                            
            loss_cl,sims = u.infoNCE_loss(imgs_out,auds_out, args,return_S=True)
            acc_v_a, acc_a_v =u.topk_accuracy(sims,k=5)

            losses.update(loss_cl.item(),B)
            acc_a_vAverage.update(acc_a_v)
            acc_v_aAverage.update(acc_v_a)
            batch_time.update(time.time() - end)
            end = time.time()


    print('Epoch: [{0}]\t Eval '
          'Loss: {loss.avg:.4f}  \t T-epoch: {t:.2f} \t'
          .format(epoch, loss=losses, t=time.time()-tic))
    return losses.avg, acc_a_vAverage.avg, acc_v_aAverage.avg

In [ ]:
# Simulate command-line arguments for loading the model
sys.argv = ['script_name', '--order_3_tensor', 
            '--simtype', 'MISA', 
            '--placesAudio', '$DATA/PlacesAudio_400k_distro/metadata/',
            '--batch_size', "32", 
            '--n_threads', '0',
            '--val_video_idx', '10']

args = get_arguments()

models = [
    {"model_name": "SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV", "epoch": 13},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-5-2ly-B32-MISA", "epoch": 100},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-5-2ly-B128-MISA", "epoch": 100},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-5-2ly-B256-MISA", "epoch": 100},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-5-2ly-B128-SISA", "epoch": 100},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-3-2ly-B128-SISA", "epoch": 39},
    {"model_name": "SSL_TIE_PlacesAudio_lr1e-4-2ly-B32-MISA", "epoch": 100},
]

# remote_path = f'/home/asantos/models/to_test/{model["model_name"]}-epoch{model["epoch"]}.pth.tar'
# u.load_model()
model_path = models[0]
model_path = f'/home/asantos/models/to_test/{model_path["model_name"]}-epoch{model_path["epoch"]}.pth.tar'

model,device = u.load_model(model_path,args)

val_dataset = PlacesAudio(args.placesAudio + 'val.json', args,mode='val')

val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False,\
        num_workers=args.n_threads, drop_last=True, pin_memory=True)

results = {}
for model_info in models:
    model_path = f'/home/asantos/models/to_test/{model_info["model_name"]}-epoch{model_info["epoch"]}.pth.tar'
    model, device = u.load_model(model_path, args)
    print(f'Validating model: {model_info["model_name"]}, Epoch: {model_info["epoch"]}')
    loss, acc_a_v, acc_v_a = validate(val_loader, model, None, device, epoch=model_info["epoch"], args=args)
    print(f'Loss: {loss:.4f}, Acc_a_v: {acc_a_v:.4f}, Acc_v_a: {acc_v_a:.4f}')
    results[model_info["model_name"]] = {
        "epoch": model_info["epoch"],
        "loss": loss,
        "acc_a_v": acc_a_v,
        "acc_v_a": acc_v_a
    }

PlacesAudio split: VAL dataset size: 1000
Validating model: SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV, Epoch: 13


100%|██████████| 31/31 [01:06<00:00,  2.15s/it]


Epoch: [13]	 Eval Loss: 3.9289  	 T-epoch: 66.63 	
Loss: 3.9289, Acc_a_v: 0.2873, Acc_v_a: 0.2157
Validating model: SSL_TIE_PlacesAudio_lr1e-5-2ly-B32-MISA, Epoch: 100


100%|██████████| 31/31 [01:07<00:00,  2.17s/it]


Epoch: [100]	 Eval Loss: 3.3159  	 T-epoch: 67.22 	
Loss: 3.3159, Acc_a_v: 0.4788, Acc_v_a: 0.4395
Validating model: SSL_TIE_PlacesAudio_lr1e-5-2ly-B128-MISA, Epoch: 100


...

Epoch: [100]	 Eval Loss: 5.3987  	 T-epoch: 69.86 	
Loss: 5.3987, Acc_a_v: 0.2752, Acc_v_a: 0.1835
Validating model: SSL_TIE_PlacesAudio_lr1e-3-2ly-B128-SISA, Epoch: 39


100%|██████████| 31/31 [01:15<00:00,  2.43s/it]


Epoch: [39]	 Eval Loss: 3.4657  	 T-epoch: 75.45 	
Loss: 3.4657, Acc_a_v: 0.1562, Acc_v_a: 0.1562
Validating model: SSL_TIE_PlacesAudio_lr1e-4-2ly-B32-MISA, Epoch: 100


100%|██████████| 31/31 [01:14<00:00,  2.39s/it]

Epoch: [100]	 Eval Loss: 3.4657  	 T-epoch: 74.19 	
Loss: 3.4657, Acc_a_v: 0.1562, Acc_v_a: 0.1562


In [8]:
import pandas as pd

df = pd.DataFrame(results)
df = df.transpose()
df.sort_values(by='loss', ascending=False, inplace=True)
df

,epoch,loss,acc_a_v,acc_v_a
SSL_TIE_PlacesAudio_lr1e-5-2ly-B128-SISA,100.0,5.398741,0.275202,0.183468
SSL_TIE_PlacesAudio_lr1e-5-2ly-B128-MISA,100.0,4.528646,0.270161,0.232863
SSL_TIE_PlacesAudio_lr1e-5-2ly-B256-MISA,100.0,4.520318,0.257056,0.221774
SSL_TIE_PlacesAudio-lr1e-5-2ly-B128-SISA-1GPUS-wV,13.0,3.928914,0.287298,0.215726
SSL_TIE_PlacesAudio_lr1e-3-2ly-B128-SISA,39.0,3.465736,0.156250,0.156250
SSL_TIE_PlacesAudio_lr1e-4-2ly-B32-MISA,100.0,3.465736,0.156250,0.156250
SSL_TIE_PlacesAudio_lr1e-5-2ly-B32-MISA,100.0,3.315941,0.478831,0.439516
